<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/data/Random_Exon_Seqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Random Genome Sequences

This notebook generates `N` random of the length `K` with the minimal distance `MIN_DIST` to any Ensembl gene.

Note that some sequences may be missing in the human genome (= consists of 'N' letters only).

## Setup

Installation for colab environment.

In [1]:
!pip install biopython pyensembl

     |████████████████████████████████| 2.3MB 2.7MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
  Created wheel for pyensembl: filename=pyensembl-1.8.7-cp36-none-any.whl size=56339 sha256=a364fc33ac98d951e97893c4eba728367ffa7b7357308e82728dcd23a8826304
  Stored in directory: /root/.cache/pip/wheels/69/8b/4d/a6c76aa39aa3619c8b8c5ff3e09cd180df57e5e9f04c655794
  Created wheel for typechecks: filename=typechecks-0.1.0-cp36-none-any.whl size=2770 sha256=9716367f52a8e6cc715c95b880771627fd8bd82a07187b5c29fb61c0d18e6237
  Stored in directory: /root/.cache/pip/wheels/27/ad/82/4a9bbc88f1d31bb1a0c053e29efafc02f56e2ed1598f5639b2
  Created wheel for datacache: filename=datacache-1.1.5-cp36-none-any.whl size=16472 sha256=efd04bf265e3c963216b42113a68d283bd3a9b5d19cb83781122ab1cac8c7c1e
  Stored in directory: /root/.cache/pip/wheels/6b/89/37/76d23043c61b5096dd0170fd6ea2f6474b38d69965b633118a
  Created wheel for memoized-property: filen

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pyensembl install --release 97 --species human

2020-06-24 08:30:20,757 - pyensembl.shell - INFO - Running 'install' for EnsemblRelease(release=97, species='homo_sapiens')
2020-06-24 08:30:20,757 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz from URL ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz
2020-06-24 08:30:20,757 - datacache.download - INFO - Downloading ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz
2020-06-24 08:30:29,213 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
2020-06-24 08:30:29,213 - datacache.download - INFO - Downloading ftp://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to /root/.

In [4]:
import pandas as pd
import numpy as np
import gzip
from tqdm.notebook import tqdm

from Bio import SeqIO   # for reading fasta files
from pyensembl import EnsemblRelease   # to get the gene list

ENSEMBL_RELEASE = 97
DNA_TOPLEVEL_FASTA_PATH = "/content/drive/My Drive/data/ensembl/Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

# to generate random sequences
N = 50_000    # how many
K = 200       # how long
MIN_DIST = 50 # how far should they be from the gene
OUTPUT_FILE = '/content/drive/My Drive/data/random/random_exons.csv'   # where to save them
# OUTPUT_FILE = 'random_seqs.csv'

CHRS = [str(chr) for chr in range(1,23)] + ['X', 'Y', 'MT']

## Get exon list

In [5]:
# release 97 uses human reference genome GRCh38
data = EnsemblRelease(ENSEMBL_RELEASE)

In [6]:
human_exons = data.exon_ids()
len(human_exons)

745513

In [12]:
human_exons[0], data.exon_by_id(human_exons[0])


('ENSE00000327880',
 Exon(exon_id='ENSE00000327880', gene_id='ENSG00000009780', gene_name='FAM76A', contig='1', start=27732603, end=27732657, strand='+'))

In [14]:
exons_full_info  = [data.exon_by_id(exon) for exon in human_exons]

In [17]:
human_exon_tuples = [(x.exon_id, x.gene_id, x.gene_name, x.contig, x.start, x.end, x.strand) for x in exons_full_info]
human_exon_table = pd.DataFrame.from_records(human_exon_tuples, columns=["id", "gene_id", "gene_symbol", "chr", "start", "end", "strand"])
assert all(human_exon_table.start <= human_exon_table.end)

human_exon_table.head()

,id,gene_id,gene_symbol,chr,start,end,strand
0,ENSE00000327880,ENSG00000009780,FAM76A,1,27732603,27732657,+
1,ENSE00000328922,ENSG00000034533,ASTE1,3,131018506,131018716,-
2,ENSE00000329326,ENSG00000129295,LRRC6,8,132583694,132583779,-
3,ENSE00000330966,ENSG00000040608,RTN4R,22,20241417,20243110,-
4,ENSE00000331106,ENSG00000063515,GSC2,22,19148576,19149095,-


In [21]:
selected_regions = human_exon_table[["chr", "start", "end"]][human_exon_table.chr.isin(CHRS) & ((human_exon_table.end - human_exon_table.start + 1) >= K)].copy()
selected_regions.chr.value_counts()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


1     23862
2     19030
3     15833
11    15300
19    15215
17    15199
12    14349
16    12533
7     12463
5     12418
6     12416
4     10514
8     10225
15     9595
14     9351
9      9196
10     9007
X      7883
20     5835
22     5618
18     4902
13     4731
21     3206
Y       917
MT       15
Name: chr, dtype: int64

In [27]:
selected_regions['length'] = selected_regions.end - selected_regions.start + 1
selected_regions['random_pos'] = [np.random.randint(c_len) for c_len in selected_regions.length]
selected_regions['random_start'] = selected_regions.start + selected_regions.random_pos
selected_regions.head()

,chr,start,end,length,end2,random_pos,random_start
1,3,131018506,131018716,211,131018517,12,131018518
3,22,20241417,20243110,1694,20242911,737,20242154
4,22,19148576,19149095,520,19148896,125,19148701
8,11,2412754,2413012,259,2412813,31,2412785
31,15,40899144,40901014,1871,40900815,290,40899434


## Random exon selection

In [28]:
sample_regions = selected_regions.sample(N)
sample_regions.shape

(50000, 7)

## Get actual genomic sequences

In [35]:
seqs = sample_regions[['chr', 'random_start']].copy().reset_index(drop=True)
seqs['random_end'] = seqs['random_start'] + K - 1
seqs['seq'] = ''

In [36]:
seqs.head()

,chr,random_start,random_end,seq
0,3,27797844,27798043,
1,4,26368050,26368249,
2,12,61709960,61710159,
3,16,85743349,85743548,
4,11,59594111,59594310,


In [37]:
def which(self):
    try:
        self = list(iter(self))
    except TypeError as e:
        raise Exception("""'which' method can only be applied to iterables.
        {}""".format(str(e)))
    indices = [i for i, x in enumerate(self) if bool(x) == True]
    return(indices)

with gzip.open(DNA_TOPLEVEL_FASTA_PATH, "rt") as handle:
    for record in tqdm(SeqIO.parse(handle, "fasta"), total=24):
        sel_seqs = which(seqs.chr == record.id)
        for i in sel_seqs:
            seqs.loc[i, "seq"] = str(record.seq[(seqs.random_start[i]-1):seqs.random_end[i]])
        
        if record.id == "MT": 
            # stop, do not read small contigs
            break

In [38]:
seqs.head()

,chr,random_start,random_end,seq
0,3,27797844,27798043,ATCAAGAAGATTTGGAGAAGGAGTCTTGGAAGCTCTGGTGAAGTTT...
1,4,26368050,26368249,ACATAGTTCAGTTCTCTGAATTGGGCATGATATGCTCTTCAGAAAT...
2,12,61709960,61710159,AAAGCCATGTGTAGTCCCACTAGCAAATGAACATAGGACCCAGGAA...
3,16,85743349,85743548,ATTCTGGGCCAAGTTTGCTATAAAAACGATGCCACTGAACCCTAAC...
4,11,59594111,59594310,TGGAGTAGGAGGACACGGTGAAAGCTGCAACATAACAGAGCTGTTC...


In [39]:
len(seqs.seq.values[0])

200

In [40]:
seqs.seq.values[1]

'ACATAGTTCAGTTCTCTGAATTGGGCATGATATGCTCTTCAGAAATCACGTTTTGAAATTACAGAAGATAAATAACCTCTCCCAATCAGTGTGTTAACTACCCTGATGATGGTTGCTTGTCTTTGTACAGATGTGTGCAGGCATACATCTTTAGTGGTGGAGGCAGTAAATGAAAACATATGGATTCAGAGTATGGAAGA'

In [48]:
any(seqs.seq.str.contains('N'))

False

## Save generated sequences to file

In [49]:
seqs.to_csv(OUTPUT_FILE, index=False)